<a href="https://colab.research.google.com/github/ChaitaliV/generative-explanation/blob/main/datacollection/unsupervised_data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Create Unsupervised Dataset for Generative Transformer
* read pdfs and save the data in .txt file.
* manually add some more data to it, from blogs and other online resources which are not downloadable.
* manually read the data once, and remove text which is not relevant
* clean the text
* create sentences
* create unsupervised learning dataset for generative transformer model by selectively masking words in all sentences

In [1]:
!pip install pyPDF2 transformers sentencepiece
!git clone https://github.com/ChaitaliV/generative-explanation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 88.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.7 MB/s eta 0:00:00
Cloning into 'generative-explanation'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 108 (delta 7), reused 3 (delta 3), pack-reused 96
Receiving objects: 100% (108/108), 6.49 MiB | 17.00 MiB/s, done.
Resolving deltas: 100% (42/42), done.


### Create Raw Dataset
* read text from the pdf which are textbooks, manuals and other web resources on depression and it's dignosis. this will be our raw data
* Go through the raw data manually, and add % to seperate topics, this should work better than seperating the text by sentences, as whole topics as input will retain long term dependencies.
* Also remove garbage text and topics that are not relevant.

In [2]:
from PyPDF2 import PdfReader
import os

In [3]:
folder_path = 'generative-explanation/datasets/unsupervised dataset'

In [4]:
def read_pdf(file_path):
    """fn to read pdf files"""
    with open(file_path, 'rb') as file:
        reader = PdfReader(file)
        text = ''
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

# get the list of all pdf from the folder
file_list = [file for file in os.listdir(folder_path) if file.endswith('.pdf')]
corpus = ''

# Iterate through PDF files and read text
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    text = read_pdf(file_path)
    corpus += text

#save the corpus as .txt file for future use
def save_to_txt(text, file_path):
    with open(file_path, 'w') as file:
        file.write(text)

save_to_txt(corpus, 'generative-explanation/raw_unsupervised_text.txt')

### Process the raw text to create Dataset with masked tokens
* load the raw data, remove numbers and special characters. and seperate the data from % token.
* mask words and create unsupervised dataset to predict the masked words.
* for masking process, mask individual masks

In [5]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import random
import numpy as np
import re
import pandas as pd

In [6]:
raw_data_path = 'generative-explanation/datasets/unsupervised dataset/raw_unsupervised_text.txt'

In [7]:
corpus = ''
with open(raw_data_path, 'r') as file:
  corpus = file.read()

In [8]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")

def mask_tokens(sentence, mask_percentage=0.4):
    """"this function will take the sentence, generate tokens,
    randomly mask 40% tokens for encoder. unmasked 60% tokens are masked in decoder.
    """
    # Tokenize the sentence
    tokenized_sentence = tokenizer(sentence, return_tensors="pt",truncation=True,max_length = 512, pad_to_max_length=True, add_special_tokens=True).input_ids[0]
    label = tokenized_sentence.clone()

    # Calculate the number of tokens to mask
    num_tokens_to_mask = int(mask_percentage * len(tokenized_sentence))

    # Randomly choose indices to mask
    encoder_masked_indices = random.sample(range(1, len(tokenized_sentence) - 1), num_tokens_to_mask)
    decoder_masked_indices = list(set(np.arange(0, len(tokenized_sentence) - 1)) - set(encoder_masked_indices))

    # Mask the chosen tokens in encoder
    for index in encoder_masked_indices:
        tokenized_sentence[index] = tokenizer.convert_tokens_to_ids(f"<masked_token>")


    #mask rest of the tokens in decoder
    for index in decoder_masked_indices:
        label[index] = tokenizer.convert_tokens_to_ids(f"<masked_token>")

    encoder_text = tokenizer.decode(tokenized_sentence.flatten())
    decoder_text = tokenizer.decode(label.flatten())

    return encoder_text, decoder_text

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the

In [9]:
def generate_data(text, split_method, masked_perc):

  #remove new line characters
  text = text.replace('\n',' ')

  #remove special characters
  pattern = re.compile('[^a-zA-Z;?.,%\']')
  clean_text = pattern.sub(' ', text)
  clean_text = clean_text.replace('  ',' ')

  #create topic sequences
  text_strings = clean_text.split(split_method)

  #create masks for each topic strings
  encoder_data = []
  decoder_data = []
  for string in text_strings:
    encoder, decoder = mask_tokens(string, mask_percentage = masked_perc)
    encoder_data.append(encoder)
    decoder_data.append(decoder)

  #create final dataframe
  df = pd.DataFrame({'Encoder': encoder_data, 'Decoder': decoder_data})
  return df

In [10]:
df1 = generate_data(corpus, '%', 0.1)
df2 = generate_data(corpus, '%', 0.3)
df3 = generate_data(corpus,'%', 0.5)
df4 = generate_data(corpus,'.',0.3)
unsupervised_dataset = pd.concat([df1,df2,df3,df4],axis=0)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input 

In [11]:
unsupervised_dataset

,Encoder,Decoder
0,Depression<unk> anxiety are different conditio...,<unk> and<unk><unk><unk><unk><unk><unk><unk><u...
1,An<unk> iety may occur as a symptom<unk> clini...,<unk> x<unk><unk><unk><unk><unk><unk><unk><unk...
2,"Do you feel irritable, isolated or withdrawn? ...",<unk><unk><unk><unk><unk><unk><unk><unk><unk><...
3,Depression signs and<unk> can differ in men an...,<unk><unk><unk> symptoms<unk><unk><unk><unk><u...
4,Other behaviors in men that could be signs of ...,<unk><unk><unk><unk><unk><unk><unk><unk><unk><...
...,...,...
1254,They<unk> wish<unk> deal with their problem<un...,<unk> may<unk> to<unk><unk><unk><unk> secret<u...
1255,These<unk> are more common than<unk> would exp...,<unk> feelings<unk><unk><unk><unk> one<unk><un...
1256,"However, every woman<unk><unk><unk> she is not...",<unk><unk><unk><unk> must realize that<unk><un...
1257,Per<unk> natal<unk> affect<unk><unk> of women ...,<unk> i<unk> Depression<unk> s thousands<unk><...


In [12]:
unsupervised_dataset.to_csv('unsupervised_dataset.csv')

##masking words instead of tokens

In [33]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")

def mask_split_words(sentence, mask_percentage=0.4):
    """"this function will take the sentence, generate tokens,
    randomly mask 40% tokens for encoder. unmasked 60% tokens are masked in decoder.
    """
    # Tokenize the sentence
    tokenized_sentence = sentence.split(' ')
    label = tokenized_sentence.copy()

    # Calculate the number of tokens to mask
    num_tokens_to_mask = int(mask_percentage * len(tokenized_sentence))


    # Randomly choose indices to mask
    encoder_masked_indices = random.choices(range(0, len(tokenized_sentence)), k=num_tokens_to_mask)
    print(len(encoder_masked_indices), len(tokenized_sentence))
    print(encoder_masked_indices)
    try:
      decoder_masked_indices = list(set(np.arange(0, len(tokenized_sentence))) - set(encoder_masked_indices))
    except:
      decoder_masked_indices = list(set(encoder_masked_indices) - set(np.arange(0, len(tokenized_sentence))))

    # Mask the chosen tokens in encoder
    for index in encoder_masked_indices:
        tokenized_sentence[index] = "<masked_token>"


    #mask rest of the tokens in decoder
    for index in decoder_masked_indices:
        label[index] = "<masked_token>"

    return tokenized_sentence, label

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [28]:
def generate_data(text, split_method, masked_perc):

  #remove new line characters
  text = text.replace('\n',' ')

  #remove special characters
  pattern = re.compile('[^a-zA-Z;?.,%\']')
  clean_text = pattern.sub(' ', text)
  clean_text = clean_text.replace('  ',' ')

  #create topic sequences
  text_strings = clean_text.split(split_method)

  #create masks for each topic strings
  encoder_data = []
  decoder_data = []
  for string in text_strings:
    encoder, decoder = mask_split_words(string, mask_percentage = masked_perc)
    encoder_data.append(encoder)
    decoder_data.append(decoder)

  #create final dataframe
  df = pd.DataFrame({'Encoder': encoder_data, 'Decoder': decoder_data})
  return df

In [34]:
df1 = generate_data(corpus, '%', 0.1)
df2 = generate_data(corpus, '%', 0.3)
df3 = generate_data(corpus,'%', 0.5)
df4 = generate_data(corpus,'.',0.3)
unsupervised_dataset_words = pd.concat([df1,df2,df3,df4],axis=0)

6 64
[22, 26, 29, 52, 34, 13]
10 107
[37, 57, 91, 5, 35, 10, 90, 5, 41, 75]
7 79
[44, 34, 76, 56, 15, 27, 19]
8 88
[72, 68, 73, 57, 86, 40, 10, 60]
10 103
[68, 17, 87, 14, 25, 58, 49, 36, 34, 79]
6 60
[46, 28, 4, 56, 47, 26]
4 48
[22, 40, 15, 23]
6 62
[5, 55, 25, 23, 59, 57]
4 48
[44, 15, 37, 20]
5 51
[5, 19, 46, 39, 19]
6 65
[26, 37, 3, 14, 13, 24]
4 49
[28, 2, 44, 24]
7 78
[34, 53, 57, 32, 65, 54, 9]
7 70
[5, 29, 61, 58, 1, 9, 30]
5 58
[14, 15, 33, 56, 15]
3 32
[8, 5, 20]
7 76
[32, 6, 1, 70, 24, 22, 31]
4 47
[13, 42, 0, 17]
4 40
[20, 13, 31, 22]
6 62
[23, 1, 41, 61, 18, 1]
6 68
[29, 61, 6, 61, 41, 21]
13 134
[69, 94, 67, 7, 98, 67, 102, 114, 44, 30, 10, 87, 108]
8 80
[54, 40, 52, 28, 11, 29, 78, 27]
4 40
[25, 0, 22, 10]
5 54
[21, 9, 39, 12, 52]
4 46
[35, 5, 40, 0]
6 60
[53, 23, 37, 11, 36, 54]
13 139
[96, 112, 102, 4, 35, 55, 105, 124, 78, 58, 8, 134, 83]
2 20
[13, 2]
3 35
[17, 11, 1]
0 2
[]
11 111
[110, 22, 41, 31, 66, 65, 62, 37, 102, 34, 64]
1 13
[2]
0 4
[]
10 105
[32, 68, 49, 70,

So unsupervised dataset is ready with encoder and decoder data input tensors.